In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
smoking = pd.read_csv('smoking_cleaned.csv')

In [3]:
smoking.head()

,gender,age,marital_status,highest_qualification,nationality,ethnicity,region,avg_salary,smoke
0,Male,38,Divorced,No Qualification,British,White,The North,3900.0,No
1,Female,42,Single,No Qualification,British,White,The North,1300.0,Yes
2,Male,40,Married,Degree,English,White,The North,32500.0,No
3,Female,40,Married,Degree,English,White,The North,13000.0,No
4,Female,39,Married,GCSE/O Level,British,White,The North,3900.0,No


In [4]:
from autogluon.tabular import TabularPredictor, TabularDataset

In [5]:
X = smoking.drop('smoke', axis=1)
y = smoking['smoke']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 45)

In [ ]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label='smoke', path=save_path).fit(smoking, presets='high_quality')

Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: agModels-predictClass/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "agModels-predictClass/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.11.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10

In [8]:
predictor = TabularPredictor.load("agModels-predictClass")

In [9]:
predictor

In [10]:
test_data = smoking.sample(100)
y_test = test_data['smoke']  # values to predict
test_data_nolab = test_data.drop(columns=['smoke'])  # delete label column to prove we're not cheating
test_data_nolab.head()

,gender,age,marital_status,highest_qualification,nationality,ethnicity,region,avg_salary
124,Female,52,Married,Higher/Sub Degree,British,White,The North,18200.0
1687,Female,49,Divorced,Other/Sub Degree,English,White,Scotland,3900.0
1380,Female,33,Married,Degree,British,Asian,South West,7800.0
6,Male,53,Married,Degree,British,White,The North,18200.0
1422,Female,43,Married,GCSE/CSE,English,White,South West,3900.0


In [70]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions( y_pred=y_pred, auxiliary_metrics=True, detailed_report=True, display=True)

Predictions:  
 124      No
1687    Yes
1380     No
6        No
1422     No
       ... 
760      No
942      No
1164     No
1025     No
871      No
Name: smoke, Length: 100, dtype: object


TypeError: TabularPredictor.evaluate_predictions() missing 1 required positional argument: 'y_true'

In [13]:
perf

{'accuracy': 0.81,
 'balanced_accuracy': 0.6022021456804065,
 'mcc': 0.36220941758254493,
 'f1': 0.3448275862068966,
 'precision': 0.8333333333333334,
 'recall': 0.21739130434782608,
 'confusion_matrix':      No  Yes
 No   76    1
 Yes  18    5,
 'classification_report': {'No': {'precision': 0.8085106382978723,
   'recall': 0.987012987012987,
   'f1-score': 0.8888888888888888,
   'support': 77.0},
  'Yes': {'precision': 0.8333333333333334,
   'recall': 0.21739130434782608,
   'f1-score': 0.3448275862068966,
   'support': 23.0},
  'accuracy': 0.81,
  'macro avg': {'precision': 0.8209219858156028,
   'recall': 0.6022021456804065,
   'f1-score': 0.6168582375478927,
   'support': 100.0},
  'weighted avg': {'precision': 0.8142198581560284,
   'recall': 0.81,
   'f1-score': 0.7637547892720307,
   'support': 100.0}}}

In [16]:
test_data.tail()

,gender,age,marital_status,highest_qualification,nationality,ethnicity,region,avg_salary,smoke
760,Female,30,Single,ONC/BTEC,British,White,Midlands & East Anglia,18200.0,No
942,Male,66,Married,Degree,British,Asian,London,13000.0,Yes
1164,Female,36,Married,GCSE/CSE,British,White,South East,1300.0,No
1025,Male,28,Single,GCSE/O Level,Refused,Refused,London,0.0,No
871,Male,40,Married,Degree,Welsh,White,London,18200.0,No


In [35]:
column = ['gender', 'age', 'marital_status', 'highest_qualification', 'nationality', 'ethnicity', 'region', 'avg_salary']

In [45]:
new_data = ['Female', 30, 'Single', 'Degree', 'Welsh', 'White', 'London', 1500]
df1 = pd.DataFrame([new_data], columns = column)

In [46]:
df1

,gender,age,marital_status,highest_qualification,nationality,ethnicity,region,avg_salary
0,Female,30,Single,Degree,Welsh,White,London,1500


In [50]:
y = predictor.predict(df1)
y

0    No
Name: smoke, dtype: object

In [52]:
predictor.feature_importance(data = smoking)

Computing feature importance via permutation shuffling for 8 features using 1691 rows with 5 shuffle sets...
	0.79s	= Expected runtime (0.16s per shuffle set)
	0.27s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
age,0.068007,0.004562,0.000002,5,0.077399,0.058615
marital_status,0.032762,0.004002,0.000026,5,0.041002,0.024521
highest_qualification,0.025547,0.003407,0.000037,5,0.032563,0.018531
avg_salary,0.024601,0.005923,0.000374,5,0.036795,0.012406
region,0.018096,0.003846,0.000231,5,0.026015,0.010177
gender,0.009817,0.002495,0.000460,5,0.014954,0.004679
nationality,0.001301,0.001794,0.090076,5,0.004994,-0.002392
ethnicity,0.000000,0.000000,0.500000,5,0.000000,0.000000


In [66]:
predictor.evaluate_predictions(y_true= df1, y_pred=y, auxiliary_metrics=True, detailed_report=True, display=True)

AssertionError: 

In [65]:
predictor.leaderboard(display=True)

                  model  score_val eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.790560    accuracy       0.000000  1.016823                0.000000           0.627726            2       True         14
1              LightGBM   0.790560    accuracy       0.000000  0.389097                0.000000           0.389097            1       True          4
2              CatBoost   0.772861    accuracy       0.000000  8.628204                0.000000           8.628204            1       True          7
3               XGBoost   0.766962    accuracy       0.015621  0.996014                0.015621           0.996014            1       True         11
4            LightGBMXT   0.764012    accuracy       0.000000  1.047141                0.000000           1.047141            1       True          3
5        NeuralNetTorch   0.764012    accuracy       0.013366  7.574719                0.013366     

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.790560,accuracy,0.000000,1.016823,0.000000,0.627726,2,True,14
1,LightGBM,0.790560,accuracy,0.000000,0.389097,0.000000,0.389097,1,True,4
2,CatBoost,0.772861,accuracy,0.000000,8.628204,0.000000,8.628204,1,True,7
3,XGBoost,0.766962,accuracy,0.015621,0.996014,0.015621,0.996014,1,True,11
4,LightGBMXT,0.764012,accuracy,0.000000,1.047141,0.000000,1.047141,1,True,3
5,NeuralNetTorch,0.764012,accuracy,0.013366,7.574719,0.013366,7.574719,1,True,12
6,NeuralNetFastAI,0.764012,accuracy,0.021815,3.934528,0.021815,3.934528,1,True,10
7,LightGBMLarge,0.758112,accuracy,0.015621,0.478631,0.015621,0.478631,1,True,13
8,KNeighborsUnif,0.758112,accuracy,0.021710,2.560985,0.021710,2.560985,1,True,1
9,ExtraTreesEntr,0.743363,accuracy,0.036534,0.474351,0.036534,0.474351,1,True,9


In [ ]:
predict